In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

df = pd.read_csv('Housing.csv')
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [2]:
# Data Preprocessing

# Changing all yes/no data to 0/1, normalization of price and area, and outlier removal 

for column in df:  
    df[column].replace(('yes','no'), (0,1), inplace = True)
    if(column == 'price' or column == 'area'):
        df[column] = (df[column] - df[column].mean()) / df[column].std()
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        df.drop(df[(df[column] < Q1-1.5*IQR) | (df[column] > Q3+1.5*IQR)].index, inplace = True) 
        
# Adding One Hot Encoding to furnishingstatus 

df = pd.get_dummies(df, columns = ['furnishingstatus']) 

df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus_furnished,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
15,2.316712,0.452182,4,1,2,0,1,0,1,1,2,1,0,1,0
16,2.316712,0.741276,4,2,2,0,0,0,1,0,1,0,0,0,1
17,2.241864,1.656740,3,2,4,0,1,1,1,0,2,1,1,0,0
18,2.204439,-0.222371,3,2,2,0,0,1,1,0,2,1,1,0,0
19,2.185727,0.654548,3,2,2,0,1,1,1,0,1,0,0,1,0


In [3]:
print(df.corr().price)
print(df.info())

price                              1.000000
area                               0.533186
bedrooms                           0.335328
bathrooms                          0.466483
stories                            0.446839
mainroad                          -0.307185
guestroom                         -0.303994
basement                          -0.190117
hotwaterheating                   -0.075979
airconditioning                   -0.471706
parking                            0.321058
prefarea                          -0.280808
furnishingstatus_furnished         0.207903
furnishingstatus_semi-furnished    0.085766
furnishingstatus_unfurnished      -0.278463
Name: price, dtype: float64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 517 entries, 15 to 544
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   price                            517 non-null    float64
 1   area           

In [4]:
# Splitting Data Set for Training and Testing

X = df.iloc[0:-1, 1:-1]
Y = df.iloc[0:-1, 0:1]
print(X.shape)
print(Y.shape)
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.7)

(516, 13)
(516, 1)


In [5]:
# Using Scikit Learn 
regr = LinearRegression()
regr.fit(x_train, y_train)
print(regr.score(x_test, y_test))

0.6950906885457309


In [6]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
y_pred = regr.predict(x_test)
mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred)
rmse = mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False)
  
print("MAE:",mae)
print("MSE:",mse)
print("RMSE:",rmse)

MAE: 0.35537150589006333
MSE: 0.20412735030376003
RMSE: 0.45180454878604315
